In [1]:
import os
os.chdir('..')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torchvision
from pytorch_lightning.callbacks import ModelCheckpoint
from src.model.lit_module import LitModule
from src.data.dataset import VideoLabelDataset
import src.constants as const
from torch.utils.data import DataLoader
from src.data.dataset import (VideoLabelDataset,
                              VideoFolderPathToTensor,
                              VideoResize)

In [4]:
checkpoint_path = './lightning_logs/version_26/checkpoints/epoch=99-step=3199.ckpt'

In [5]:
model = LitModule.load_from_checkpoint(checkpoint_path)

In [6]:
dataset = VideoLabelDataset(
            const.LABELS_TABLE_QA_PATH,
            img_transform=torchvision.transforms.Compose([
                VideoFolderPathToTensor(),
                VideoResize(const.IMG_SIZE)]))

In [7]:
dataloader = DataLoader(dataset, batch_size=10, num_workers=2)

In [8]:
videos, questions, answers, hidden_states  = iter(dataloader).next()

In [9]:
predictions = model.eval()(videos)
predictions

tensor([[543.4559, 996.8821,  37.2706],
        [543.4633, 996.8958,  37.2711],
        [543.4277, 996.8304,  37.2686],
        [543.3871, 996.7563,  37.2658],
        [543.3900, 996.7614,  37.2661],
        [543.3262, 996.6446,  37.2617],
        [543.4123, 996.8020,  37.2676],
        [543.3502, 996.6880,  37.2633],
        [543.4186, 996.8138,  37.2680],
        [543.4409, 996.8543,  37.2695]], grad_fn=<AddmmBackward>)

In [10]:
hidden_states

tensor([[ 772.6472,  912.8349,   36.0000],
        [ 627.9586, 1077.1292,   25.0000],
        [ 472.4544, 1046.5730,   47.0000],
        [ 430.9703, 1006.6889,   30.0000],
        [ 516.7844, 1069.5571,   38.0000],
        [ 437.3075,  983.9614,   44.0000],
        [ 711.1207, 1024.2825,   37.0000],
        [ 389.1723,  911.7273,   48.0000],
        [ 618.8253, 1055.9926,   39.0000],
        [ 560.6606,  916.3023,   40.0000]], dtype=torch.float64)

In [32]:
mse_loss = torch.nn.MSELoss(reduction='sum')
mse_hidden = mse_loss(predictions[0:2,:].type(torch.float32),
                      hidden_states[0:2,:].type(torch.float32))
mse_hidden

tensor(73321.6484, grad_fn=<MseLossBackward>)

In [33]:
((772.6472 - 543.4424)**2 + (912.8349-996.8575)**2 + (36.0000-37.2697)**2 + \
(627.9586-543.4633)**2 + (1077.1292-996.8958)**2 + (25.0000-37.2711)**2)

73323.68388475002

In [ ]:
model.eval()(videos)

In [22]:
predictions[0:2,:]

tensor([[543.4559, 996.8821,  37.2706],
        [543.4633, 996.8958,  37.2711]], grad_fn=<SliceBackward>)